<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/diffusion/InpaintingAnything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
%cd /content
 
!git clone https://github.com/geekyutao/Inpaint-Anything.git
 
%cd /content/Inpaint-Anything

In [ ]:
# torchtext.legacy solve: https://github.com/Stability-AI/stablediffusion/issues/8
%cd /content/Inpaint-Anything
 
!pip uninstall yellowbrick imbalanced-learn -y
!python -m pip install -e segment_anything
!python -m pip install pyyaml tqdm easydict==1.9.0 scikit-image scikit-learn albumentations==0.5.2 hydra-core==1.1.0 pytorch-lightning==1.8.3.post0 tabulate kornia==0.5.0 webdataset packaging wldhx.yadisk-direct
!python -m pip install yellowbrick imbalanced-learn
!pip install huggingface-hub

Download pretrained models

In [ ]:
%cd /content/Inpaint-Anything
 
!mkdir -p pretrained_models
 
!wget -c https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth \
      -O ./pretrained_models/sam_vit_h_4b8939.pth
 
# ダウンロード不安定なためHuggingFace使用
# !curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
# !unzip big-lama.zip
# !mv big-lama ./pretrained_models
!mkdir -p pretrained_models/big-lama/models
!wget -c https://huggingface.co/camenduru/big-lama/resolve/main/big-lama/models/best.ckpt \
      -O pretrained_models/big-lama/models/best.ckpt
!wget -c https://huggingface.co/camenduru/big-lama/raw/main/big-lama/config.yaml \
      -O pretrained_models/big-lama/config.yaml

Import Library

In [3]:
import cv2
from google.colab.patches import cv2_imshow
 
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
point_x = 58 #@param {type:"slider", min:0, max:800, step:1}
point_y = 688 #@param {type:"slider", min:0, max:800, step:1}
 
img = cv2.imread('/content/Inpaint-Anything/example/remove-anything/person.jpg')
 
cv2.circle(
    img,  
    (point_x, point_y), # 座標
    5,                  # 円の半径 
    (0,0,255),          # カラー
    -1                  # 太さ
    ) 
 
cv2_imshow(img)

In [ ]:
%cd /content/Inpaint-Anything
 
!python remove_anything.py \
    --input_img /content/Inpaint-Anything/example/remove-anything/person.jpg \
    --point_coords {point_x} {point_y} \
    --coords_type key_in \
    --point_labels 1 \
    --dilate_kernel_size 15 \
    --output_dir ./results \
    --sam_model_type "vit_h" \
    --sam_ckpt ./pretrained_models/sam_vit_h_4b8939.pth \
    --lama_config ./lama/configs/prediction/default.yaml \
    --lama_ckpt ./pretrained_models/big-lama/

In [ ]:
# display results
from PIL import Image
display(Image.open("/content/Inpaint-Anything/results/person/inpainted_with_mask_2.png"))

Replace Anything

In [ ]:
!python -m pip install diffusers transformers accelerate scipy safetensors

In [ ]:
%cd /content/Inpaint-Anything

!python fill_anything.py \
    --input_img ./example/fill-anything/sample1.png \
    --coords_type key_in \
    --point_coords 750 500 \
    --point_labels 1 \
    --text_prompt "a teddy bear on a bench" \
    --dilate_kernel_size 50 \
    --output_dir ./results \
    --sam_model_type "vit_h" \
    --sam_ckpt ./pretrained_models/sam_vit_h_4b8939.pth

In [ ]:
display(Image.open("/content/Inpaint-Anything/example/fill-anything/sample1.png"))

In [ ]:
display(Image.open("/content/Inpaint-Anything/results/sample1/filled_with_mask_1.png"))